In [2]:
%load_ext autoreload
%autoreload 2

import sympy as sm
import sympy.physics.mechanics as me
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp
from semiflex import SemiFlexBody
from force_controlers import generator_torque_control,rotor_torque_jit,thrust_force_jit
from utils import smaa
me.init_vprinting(use_latex="mathjax")

In [3]:
t = me.dynamicsymbols._t
# masses
m_nac, m_rot = sm.symbols("m_nac,m_rot")
# distances
xn_cm, zn_cm, h_t, h_t_cm, d_nac_rot = sm.symbols("xn_cm, zn_cm,h_t, h_t_cm,d_nr")

# fixed angles
alpha_tilt = sm.symbols("alpha")
z, g, c_quad = sm.symbols("z, g, c_quad")

# Inertias
Ixx_n, Iyy_n, Izz_n, Ixx_r, Iyy_r, Izz_r = sm.symbols(
    "Ixx_n,Iyy_n,Izz_n,Ixx_r,Iyy_r,Izz_r"
)
# Torques and Forces on the hub
fx,fy,fz,mx,my,mz,torque_gen = sm.symbols("Fx,Fy,Fz,Mx,My,Mz,T_g")

q5, u5 = me.dynamicsymbols("q5,u5")

E = me.ReferenceFrame("E")  # Earth
## Points
og = me.Point("O")
og.set_vel(E, 0)

tower = SemiFlexBody(E,og,'tower',zcm=h_t_cm,dof_x=2,dof_y=2)
q1,q2,q3,q4 = tower.q
u1,u2,u3,u4 = tower.u

N = me.ReferenceFrame("N")  # Nacelle, rotated by tower deformations
N.orient_body_fixed(E, (tower.theta_y, tower.theta_x, 0), "XYZ")

# Rotor, tilted by a fixed amount and rotated by azimuth - normal rotor motion
H = me.ReferenceFrame("H") #hub
H.orient_body_fixed(N, (0, alpha_tilt, 0), "XYZ")
# R.orient_body_fixed(N, (alpha_tilt, q5, 0), "YXZ")
# R.set_ang_vel(R, u5 * R.x)
R = me.ReferenceFrame("R")
R.orient_body_fixed(H, (q5,0, 0), "XYZ")
R.set_ang_vel(N, u5 * N.x)

nacelle_cm = og.locatenew(
    "nac_cm", h_t * E.z + zn_cm*N.z + xn_cm*N.x + (q1 + q2)*E.x  + (q3 + q4) * E.y
)
rotor_cm = nacelle_cm.locatenew("rot_cm", -d_nac_rot * R.x)

I_nacelle = me.inertia(N, Ixx_n, Iyy_n, Izz_n)
I_rotor = me.inertia(R, Ixx_r, Iyy_r, Izz_r)

nacelle = me.RigidBody("nacelle", nacelle_cm, N, m_nac, (I_nacelle, nacelle_cm))
rotor = me.RigidBody("rotor", rotor_cm, R, m_rot, (I_rotor, rotor_cm))

v_rot_x = rotor_cm.vel(E).dot(E.x) # rotor x velocity
quad_damp_force = sm.simplify(-c_quad * sm.Abs(v_rot_x) * v_rot_x)

# 2) your existing loads (gravity):
loads = [
    # (tower_cm,   -m_t * g * E.z),
    (rotor_cm, quad_damp_force * E.x), # quad force
    (rotor_cm, -m_rot * g * E.z),
    (nacelle_cm, -m_nac * g * E.z),
    (rotor_cm, fx * H.x),
    (rotor_cm, fy * H.y),
    (rotor_cm, fz * H.z),
    (R, mx * R.x),
    (H, my * H.y),
    (H, mz * H.z),
    (R, -torque_gen * R.x),
    (N, torque_gen * R.x), 
]

## add tower contribution to loads and bodies
bodies =  [nacelle, rotor] + tower.get_bodies()
loads = loads + tower.loads

coordinates = [q1, q2, q3, q4, q5]
speeds = [u1, u2, u3, u4, u5]

kin_eqs = [
    q1.diff(t) - u1,
    q2.diff(t) - u2,
    q3.diff(t) - u3,
    q4.diff(t) - u4,
    q5.diff(t) - u5,
]
nacelle_cm.v1pt_theory(og,E,E)
# # 5. Set up Kane’s method
KM = me.KanesMethod(E, q_ind=coordinates, u_ind=speeds, kd_eqs=kin_eqs)

# # # 4) form the equations
fr, frstar = KM.kanes_equations(bodies=bodies, loads=loads)

/home/lorenzo/miniconda3/envs/openfast/lib/python3.12/site-packages/sympy/physics/vector/point.py:577: UserWarning: 
Velocity of nac_cm automatically calculated based on point O but it is
also possible from points(s): [rot_cm]. Velocities from these points
are not necessarily the same. This may cause errors in your
calculations.
  warn(filldedent(f"""


In [4]:
mass_full = sm.trigsimp(KM.mass_matrix_full)
forcing_full = KM.forcing_full
state_syms = [q1,q2,q3,q4,q5,u1,u2,u3,u4,u5]
ext_forces = [fx,fy,fz,mx,my,mz,torque_gen]
# ext_forces = [force_wind,torque_wind,torque_gen]

In [5]:
rotor.masscenter.pos_from(og)

-dₙᵣ r_x + (q₁ + q₂) e_x + (q₃ + q₄) e_y + hₜ e_z + xn_cm n_x + zn_cm n_z

In [6]:
mass_full[-1,-1]

        2              2       2               2       2          2         2   
Ixxᵣ⋅cos (α) + Iyyᵣ⋅sin (α)⋅sin (q₅) + Izzᵣ⋅sin (α)⋅cos (q₅) + dₙᵣ ⋅mᵣₒₜ⋅sin (α)

In [7]:
all_free_symbols = (KM.mass_matrix_full.free_symbols | KM.forcing_full.free_symbols) - set(ext_forces +[t])

sorted_syms = sorted([str(x) for x in all_free_symbols])
print({k: 0.0 for k in sorted_syms}) # to create a placeholder for subs_num

{'Ixx_n': 0.0, 'Ixx_r': 0.0, 'Iyy_n': 0.0, 'Iyy_r': 0.0, 'Izz_n': 0.0, 'Izz_r': 0.0, 'alpha': 0.0, 'c_quad': 0.0, 'c_xTO_1': 0.0, 'c_xTO_2': 0.0, 'c_yTO_1': 0.0, 'c_yTO_2': 0.0, 'd_nr': 0.0, 'g': 0.0, 'k_xTO_1': 0.0, 'k_xTO_2': 0.0, 'k_yTO_1': 0.0, 'k_yTO_2': 0.0, 'm_nac': 0.0, 'm_rot': 0.0, 'm_xTO_1': 0.0, 'm_xTO_2': 0.0, 'm_yTO_1': 0.0, 'm_yTO_2': 0.0, 'r_xTO_1': 0.0, 'r_xTO_2': 0.0, 'r_yTO_1': 0.0, 'r_yTO_2': 0.0, 'xn_cm': 0.0, 'zn_cm': 0.0}


In [8]:
import pickle

results_to_save = {
    'mass_full': mass_full,
    'forcing_full': forcing_full,
    'mass_sma': smaa(smaa(mass_full,tower.theta_y),tower.theta_x),
    'forcing_sma':smaa(smaa(forcing_full,tower.theta_y),tower.theta_x),
    'coordinates' : coordinates,
    'speeds' : speeds,
    'ext_forces': ext_forces
}

with open('5dof_km_res.pkl', 'wb') as f:
    pickle.dump(results_to_save, f)

# --- LOAD FROM FILE ---
with open('5dof_km_res.pkl', 'rb') as f:
    loaded_results = pickle.load(f)

In [9]:
freesyms = forcing_full.free_symbols | mass_full.free_symbols

freesyms-tower.free_symbols # actual free symbols outside the tower

{Fx, Fy, Fz, Ixxₙ, Ixxᵣ, Iyyₙ, Iyyᵣ, Izzₙ, Izzᵣ, Mx, My, Mz, T_g, α, c_quad, d ↪

↪ ₙᵣ, g, m_nac, mᵣₒₜ, t, xn_cm, zn_cm}

In [56]:
## results from combination of masses stiffness and shape functions
k1, k2 = 80e5, 2436e7
mt1,mt2 =  710_000, 3320_000
r_1t = 0.0185
r_2t = 0.268
# c_damp = 330_000

c_damp1 = 0.01*2*np.sqrt(mt1*k1)
c_damp2 = 0.02*2*np.sqrt(mt2*k2)
 ## this is how we fill in the tower variables as a subs dict
t_subs,_tm = tower.get_subs_dict()

#Generalized masses for mode shapes
t_subs[_tm['m_xTO_1']] = mt1 # Fore- aft 1
t_subs[_tm['m_xTO_2']] = mt2
t_subs[_tm['m_yTO_1']] = mt1
t_subs[_tm['m_yTO_2']] = mt2

t_subs[_tm['k_xTO_1']] = k1
t_subs[_tm['k_xTO_2']] = k2
t_subs[_tm['k_yTO_1']] = k1
t_subs[_tm['k_yTO_2']] = k2

t_subs[_tm['r_xTO_1']] = r_1t
t_subs[_tm['r_xTO_2']] = r_2t
t_subs[_tm['r_yTO_1']] = r_1t
t_subs[_tm['r_yTO_2']] = r_2t

t_subs[_tm['c_xTO_1']] = c_damp1
t_subs[_tm['c_xTO_2']] = c_damp2
t_subs[_tm['c_yTO_1']] = c_damp1
t_subs[_tm['c_yTO_2']] = c_damp2

subs_dims = {
    xn_cm: 1.9,
    zn_cm: 1.75,
    d_nac_rot: 5,
    alpha_tilt: np.deg2rad(5),
    m_rot: 110_0000,
    m_nac: 240_0000,
    # Nacelle inertias (rectangular prism approx)
    Ixx_n: 3.23e6,  # kg·m²
    Iyy_n: 3.07e6,  # kg·m²
    Izz_n: 2.61e6,  # kg·m²
    # Rotor (hub sphere approx)
    Ixx_r: 1.93e5,  # kg·m²
    Iyy_r: 1.93e5,  # kg·m²
    Izz_r: 1.93e5,  # kg·m²
    g: 9.81,
}
# 115926
# 2.60789E+06

In [ ]:
# 1. Substitute values

# 2. Lambdify for fast evaluation


M_num = mass_full.subs(subs_dims).subs(t_subs)
F_num = forcing_full.subs(subs_dims).subs(t_subs)
# M_num = mass_full.subs(subs_dims).subs(t_subs)
# F_num = forcing_full.subs(subs_dims).subs(t_subs)

M_num_sma = smaa(smaa(mass_full,tower.theta_y),tower.theta_x).subs(subs_dims).subs(t_subs)
F_num_sma = smaa(smaa(forcing_full,tower.theta_y),tower.theta_x).subs(subs_dims).subs(t_subs)

#numexpr, cython, numpy

M_func = sm.lambdify(state_syms, M_num, 'numpy')
F_func = sm.lambdify(state_syms+ext_forces, F_num, 'numpy')

M_func_sma = sm.lambdify(state_syms, M_num_sma, 'numpy')
F_func_sma = sm.lambdify(state_syms+ext_forces, F_num_sma, 'numpy')

In [6]:
R = 50
omega_max = 12

def rhs(t, y,ws=0):
    omega = y[9]
    f_wind = thrust_force_jit(R,ws)
    t_wind = rotor_torque_jit(R,ws,omega)
    t_gen = generator_torque_control(t_wind,omega,omega_max,k_p=1)
    M = M_func(*y)
    F = F_func(*y,f_wind,t_wind,t_gen).flatten()
    return np.linalg.solve(M, F)

# def rhs_sma(t, y,ws=0):
#     omega = y[9]
#     f_wind = thrust_force_jit(R,ws)
#     t_wind = rotor_torque_jit(R,ws,omega)
#     t_gen = generator_torque_control(t_wind,omega,omega_max,k_p=0.1)
#     M = M_func_sma(*y)
#     F = F_func_sma(*y,f_wind,t_wind,t_gen).flatten()
#     return np.linalg.solve(M, F)